In [1]:
import numpy as np
import scipy.stats as st

In [2]:
# Constants
CONSTANT_lL = 0.15
CONSTANT_Ld95 = 30

CONSTANT_ma = 0.015
CONSTANT_mr = 4.2
CONSTANT_mt = 2.75

CONSTANT_Cf = 0.5
CONSTANT_Cg = 0.25

In [23]:
def get_l_factor(L=CONSTANT_lL, d95=CONSTANT_Ld95):
    """Returns the l factor

    Args:
        L (float, optional): liberation size in cm, 150 microns from p80 of rougher flotation feed. 
        Assume as p80 of the rougher floation feed. Defaults to 0.15.
        d95 (int, optional): 95% passing in cm. Bern estimates 30 cm - 100 cm. Defaults to 30.

    Returns:
        float: l_factor
    """
    return np.sqrt(L / d95)

def get_m_factor(a=CONSTANT_ma, r=CONSTANT_mr, t=CONSTANT_mt):
    """Returns the m factor

    Args:
        a (float, optional): average grade as a decimal. Defaults to 0.015 (1.5%).
        r (float, optional): mineral density. Use the density for chalcopyrite. Defaults to 4.2.
        t (float, optional): gangue density. From the Minesense Amenability Study Table 1. Defaults to 2.75.

    Returns:
        _type_: _description_
    """
    return (1 - a) / a * ((1 - a) * r + a * t)

def get_C_factor(f=CONSTANT_Cf, g=CONSTANT_Cg, l=get_l_factor(), m=get_m_factor()):
    """returns C factor from params f, g, l, m

    Args:
        f (float, optional): f factor, 0.5 for non-gold. 0.2 for gold. Defaults to 0.5.
        g (float, optional): g factor, as size ration increases, g decreases. Defaults to 0.25.
        l (float, optional): l factor. Defaults to get_l_factor().
        m (float, optional): m factor. Defaults to get_m_factor().

    Returns:
        float: returns the C factor
    """
    return f * g * l * m

In [4]:
get_C_factor()

2.4251265623877516

In [5]:
Ms = 30 # kg
variation = get_C_factor() / Ms
sigma = np.sqrt(variation)

In [6]:
sigma

0.2843194542756294

In [7]:
1.5 * (sigma / 2)  

# Why did I do this

0.21323959070672205

In [8]:
st.norm.cdf(sigma)

0.61191721064182

In [35]:
def sigmas_from_conf(conf):
    """returns the number of standard deviations from the mean with a given confidence interval
    i.e. with a 95% conf interval (conf = 0.95), function will return 1.96, meaning a 95% confidence that the
    value is at most 1.96 stds (Z) away from the mean.

    Args:
        conf (float): confidence interval as a decimal

    Returns:
        float: number of stds away from the mean the actual value is away from the calculated value within the given confindence interval
    """
    return st.norm.ppf(1 - (1 - conf) / 2)

def sigma_from_params(conf_int, conf_range, grade):
    """Returns the value of sigma given a confidence interval, range, and average grade. This value can be fed into other 
    formulas to determine the minimum sample size.

    Args:
        conf_int (float): Confidence interval as a decimal percentage: 95% is 0.95
        conf_range (float): Range for the given confidence interval: 95% confident that the value is within 0.2% of the true value
        grade (float): Average grade of the distribution

    Returns:
        float: A sigma value: sigma = 0.01 (i.e. 1% error)
    """
    return (conf_range / grade) / sigmas_from_conf(conf_int)

def sample_mass_from_params(C, d95, sigma):
    """Return the minimum sample mass (in grams) to achieve the required degree of confidence & other errors associated

    Args:
        C (float): C value in g/cm^3
        d95 (float): 95% passing size of the distribution in cm
        sigma (float): sigma value

    Returns:
        _type_: _description_
    """
    return C * d95**3 / sigma**2

In [10]:
sigmas_from_conf(0.95)

1.959963984540054

In [11]:
d = .30
(get_C_factor() * (d**3)) / (sigmas_from_conf(0.90) ** 2)

0.0242015766131399

In [12]:
(get_C_factor() * 30**3) / sigmas_from_conf(0.90)

39808.05107006709

# Sample Calculation

Consider a copper ore, assaying ~ 1.5 % Cu, which must be routinely sampled on crusher product for assay to a 90% confidence level of +/- 0.1% Cu. 


In [47]:
# Solution
# Relative precision sigma in relative terms is calculated from :
#    X * sigma {or the confidence interval, x = ~2 if confidendence interval is 95%} = deviation from mean / mean percentage
# 1.96 sigma = 0.1 % / 1.5% in the example above

confidence_level = 0.90 # * 100%
confidence_range = 0.1 # %
average_grade = 1.5 # %

sigma = (confidence_range / average_grade) / sigmas_from_conf(confidence_level)
sigma

0.04053045546078461

In [49]:
d95 = 30
C = get_C_factor(
    f=0.5,
    g=0.25,
    l=get_l_factor(
        L=0.15,
        d95=d95
    ),
    m=get_m_factor(
        a=0.015,
        r=4.2,
        t=2.75
    )
)

mass_in_grams = sample_mass_from_params(C, d95, sigma)
print(f'{mass_in_grams / 1000:0.2f} kgs')

39859.81 kgs


In [37]:
d95 = 2.5
sigma = sigma_from_params(0.95, 0.1, 5)
C = get_C_factor(
    f=0.5,
    g=0.25,
    l=get_l_factor(
        L=0.015,
        d95=d95
    ),
    m=get_m_factor(
        a=0.058,
        r=7.6,
        t=2.65
    )
)
mass_in_grams = sample_mass_from_params(C, d95, sigma)
mass_in_grams / 1000

172.56566877250697